In [5]:
import pandas as pd
from termcolor import colored, cprint
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.cluster import KMeans
from scipy.spatial import distance

In [6]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

2023-02-02 12:59:32.352398: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-02 12:59:32.352437: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-02 12:59:32.352463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-9bd2d7): /proc/driver/nvidia/version does not exist
2023-02-02 12:59:32.352704: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 12:59:33.287589: W tensorflow/tsl/framework/cpu

In [28]:
data = pd.read_csv('/workspaces/codespaces-jupyter/data/test_data.csv')
# sentences = data['text'].values

# sentence_embeddings = model(sentences)
sentence_embeddings = model(data['text'].values)

In [29]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(sentence_embeddings)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [30]:
centroids = kmeans.cluster_centers_

In [55]:
results = []
for i, embedding in enumerate(sentence_embeddings):
    centroid = centroids[kmeans.labels_[i]]
    dist = distance.euclidean(embedding, centroid)
    cluster = kmeans.labels_[i]
    result = {'movie_id': data['movie_id'][i], 'cluster': cluster, 'distance_from_centroid': dist}
    results.append(result)

In [60]:
df = pd.DataFrame(results)
cprint("============================================================================================================================================================","cyan")
print(df)
cprint("============================================================================================================================================================","yellow")

    movie_id  cluster  distance_from_centroid
0        163        8                0.617964
1        393        2                0.689873
2        416        5                0.589512
3        420        7                0.607917
4        459        2                0.633172
..       ...      ...                     ...
92      8426        2                0.749626
93      8509        1                0.912565
94      8637        9                0.502169
95      8727        2                0.663614
96      8844        2                0.664410

[97 rows x 3 columns]


In [61]:
# header = ["movie_id", "cluster", "distance_from_centroid"]
df.to_csv('/workspaces/codespaces-jupyter/data/result.csv',index=False)